# 7_SCIKIT-LEARN

## 7.1. Основы работы с классами, строящими модели предварительной подготовки данных и модели машинного обучения

Scikit-learn использует объектно-ориентированный подход, где каждая модель — это класс с определенными методами. Давайте разберемся, как это работает.